#### IMPORTAMOS LAS LIBRERIAS NECESARIAS

In [2]:
import pandas as pd
import warnings 
import ast
import numpy as np
import gdown
warnings.filterwarnings("ignore")

In [ ]:
credits_file_url = "https://drive.google.com/uc?id=10ESugAZdfsURSVUk2vv7MpyJ_PS2YZIA"
movies_file_url = "https://drive.google.com/uc?id=1ykRh8AgzSWqecZSBGqMHkUanvRhRiTMD"

In [ ]:
credits_output_path = "./Datasets/"
movies_output_path = "./Datasets/"

In [ ]:
gdown.download(credits_file_url, credits_output_path, quiet=False)
gdown.download(movies_file_url, movies_output_path, quiet=False)

##### INGESTA DE LOS DATOS

In [3]:
credits = pd.read_csv(".\Datasets\credits.csv")
df = pd.read_csv(".\Datasets\movies_dataset.csv")

##### Primer aproximacion a los datos, vemos su composicion, sus datatypes, etc

In [4]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [5]:
credits.info() # Observamos que no hay valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [6]:
credits.iloc[1] # Vemos como esta compuesta la columna anidad

cast    [{'cast_id': 1, 'character': 'Alan Parrish', '...
crew    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
id                                                   8844
Name: 1, dtype: object

##### Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [7]:
# En la API solo se nos pide actores de la columna cast por lo que extraemos solo los valores que pertenezcan a la llave "name"

def nombre_actores(fila): # Definimos la funcion
    try:                  
        a = ast.literal_eval(fila["cast"]) # Mediante la libreria ast conseguimos evaluar cual es el datatype de los valores (ya que son strings) y los ingestamos en la variable "a" de actores
        if len(a) == 0: # Chequeamos que la lista no este vacia
            return None # De lo contrario devolvemos None
        nombre_actor = [actor.get("name") for actor in a] # Por cada actor evaluado en cada lista de diccionarios de cada fila retiramos solo su nombre
        return ", ".join(nombre_actor) # Devolvemos los nombres de los actores separados por coma
    except Exception: 
        return None # Si hay algun error devolvemos None


credits["actores"] = credits.apply(nombre_actores, axis=1) # Aplicamos la funcion al dataframe y creamos la columna "actores" 

credits.drop(columns="cast",inplace=True) # Dropeamos la columna original

In [8]:
# Chequeamos como quedaron los datos limpios y como esta compuesta la proxima columna a desanidar
credits

,crew,id,actores
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ..."
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph..."
3,"[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,"Whitney Houston, Angela Bassett, Loretta Devin..."
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"Steve Martin, Diane Keaton, Martin Short, Kimb..."
...,...,...,...
45471,"[{'credit_id': '5894a97d925141426c00818c', 'de...",439050,"Leila Hatami, Kourosh Tahami, Elham Korda"
45472,"[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109,"Angel Aquino, Perry Dizon, Hazel Orencio, Joel..."
45473,"[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758,"Erika Eleniak, Adam Baldwin, Julie du Page, Ja..."
45474,"[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,"Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav..."


In [9]:
# Comprobamos como esta compuesto el dato y cual es el relevante
d = ast.literal_eval(credits.crew.iloc[0])
d[0]["job"]

'Director'

In [10]:
# Usando una funcion similar a la anterior conseguimos los datos de la llave "name" solo cuando el valor de la llave "job" 
# del mismo diccionario del que es extraido sea "Director"

def nombre_director(fila):
    try:
        directores = ast.literal_eval(fila["crew"])
        directores_nombre = [e["name"] for e in directores if e["job"] == "Director"]
        return ", ".join(directores_nombre)
    except (SyntaxError, ValueError):
        return None

credits["director"] = credits.apply(nombre_director, axis=1)

credits.drop(columns="crew",inplace=True)


In [11]:
# Columna terminada

credits

,id,actores,director
0,862,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,8844,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,15602,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,31357,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,11862,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer
...,...,...,...
45471,439050,"Leila Hatami, Kourosh Tahami, Elham Korda",Hamid Nematollah
45472,111109,"Angel Aquino, Perry Dizon, Hazel Orencio, Joel...",Lav Diaz
45473,67758,"Erika Eleniak, Adam Baldwin, Julie du Page, Ja...",Mark L. Lester
45474,227506,"Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav...",Yakov Protazanov


In [12]:
# Vemos la composicion de la tabla de las peliculas
df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [13]:
# Chequeamos nulos, datatype, etc
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [14]:
# Comenzamos con la limpieza de las columnas
df["belongs_to_collection"]

0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
45461                                                  NaN
45462                                                  NaN
45463                                                  NaN
45464                                                  NaN
45465                                                  NaN
Name: belongs_to_collection, Length: 45466, dtype: object

In [15]:
# Los valores nulos los reemplazamos por "None" para que la funcion "literal_eval" de la libreria ast funcione correctamente
df["belongs_to_collection"].fillna("None",inplace=True)

In [16]:
# Usando la misma funcion que la de los actores pero cambiando los parametros conseguimos los valores que queremos
def colleciones(fila):
    try:    
        bc = ast.literal_eval(fila["belongs_to_collection"])
        collection = bc.get("name")
        return collection
    except Exception:
        return None
    
df["collecion"] = df.apply(colleciones, axis=1)

df.drop(columns="belongs_to_collection",inplace=True)

In [17]:
df

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collecion
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,None
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,None
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,...,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,None
45462,False,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,...,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,None
45463,False,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,...,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,None
45464,False,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,...,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,None


In [18]:
# Misma historia. Funcion similar a la de actores pero cambiando parametros
def generos(fila): 
    try:
        g = ast.literal_eval(fila["genres"])
        if len(g) == 0:
            return None 
        nombre_genero = [genero.get("name") for genero in g]
        return ", ".join(nombre_genero)
    except Exception:
        return None


df["generos"] = df.apply(generos, axis=1)

df.drop(columns="genres",inplace=True)

In [19]:
df

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collecion,generos
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,"Animation, Comedy, Family"
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,None,"Adventure, Fantasy, Family"
2,False,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection,"Romance, Comedy"
3,False,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,None,"Comedy, Drama, Romance"
4,False,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection,Comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,0,http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,...,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,None,"Drama, Family"
45462,False,0,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,...,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,None,Drama
45463,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,None,"Action, Drama, Thriller"
45464,False,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0,None,None


In [20]:
# Devuelta.
def idioma(fila): 
    try:
        i = ast.literal_eval(fila["spoken_languages"])
        if len(i) == 0:
            return None 
        nombre_idioma = [idioma.get("name") for idioma in i]
        return ", ".join(nombre_idioma)
    except Exception:
        return None
    
df["idioma"] = df.apply(idioma, axis=1)

df.drop(columns="spoken_languages",inplace=True)


In [21]:
df

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,runtime,status,tagline,title,video,vote_average,vote_count,collecion,generos,idioma
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,81.0,Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,"Animation, Comedy, Family",English
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,None,"Adventure, Fantasy, Family","English, Français"
2,False,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection,"Romance, Comedy",English
3,False,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,None,"Comedy, Drama, Romance",English
4,False,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection,Comedy,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,0,http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,...,90.0,Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,None,"Drama, Family",فارسی
45462,False,0,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,...,360.0,Released,NaN,Century of Birthing,False,9.0,3.0,None,Drama,
45463,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,90.0,Released,A deadly game of wits.,Betrayal,False,3.8,6.0,None,"Action, Drama, Thriller",English
45464,False,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,87.0,Released,NaN,Satan Triumphant,False,0.0,0.0,None,None,None


In [22]:
# Chequeamos los nombres de las columnas que no son visibles debido a la longitus del DataFrame
df.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'collecion', 'generos', 'idioma'],
      dtype='object')

In [23]:
# Visualizamos aquellas columnas escondidas y encontramos mas columnas anidadas
df[["production_companies","production_countries","release_date","revenue"]]

,production_companies,production_countries,release_date,revenue
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0
1,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0
2,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0
3,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0
4,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0
...,...,...,...,...
45461,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0
45462,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0
45463,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0
45464,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0


In [24]:
# Utilizamos la misma funcion en ambos casos para conseguir los valores requeridos
def compania(fila): 
    try:
        c = ast.literal_eval(fila["production_companies"])
        if len(c) == 0:
            return None 
        companias = [compania.get("name") for compania in c]
        return ", ".join(companias)
    except Exception:
        return None


df["compania_productora"] = df.apply(compania, axis=1)

df.drop(columns="production_companies",inplace=True)

In [25]:
def pais_produccion(fila): 
    try:
        pc = ast.literal_eval(fila["production_countries"])
        if len(pc) == 0:
            return None 
        paises = [pais.get("name") for pais in pc]
        return ", ".join(paises)
    except Exception:
        return None


df["pais_produccion"] = df.apply(pais_produccion, axis=1)

df.drop(columns="production_countries",inplace=True)

In [26]:
# DataFrame desanidado:
df

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,tagline,title,video,vote_average,vote_count,collecion,generos,idioma,compania_productora,pais_produccion
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,"Animation, Comedy, Family",English,Pixar Animation Studios,United States of America
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,None,"Adventure, Fantasy, Family","English, Français","TriStar Pictures, Teitler Film, Interscope Com...",United States of America
2,False,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection,"Romance, Comedy",English,"Warner Bros., Lancaster Gate",United States of America
3,False,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,None,"Comedy, Drama, Romance",English,Twentieth Century Fox Film Corporation,United States of America
4,False,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection,Comedy,English,"Sandollar Productions, Touchstone Pictures",United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,0,http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,...,Rising and falling between a man and woman,Subdue,False,4.0,1.0,None,"Drama, Family",فارسی,None,Iran
45462,False,0,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,...,NaN,Century of Birthing,False,9.0,3.0,None,Drama,,Sine Olivia,Philippines
45463,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,A deadly game of wits.,Betrayal,False,3.8,6.0,None,"Action, Drama, Thriller",English,American World Pictures,United States of America
45464,False,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,NaN,Satan Triumphant,False,0.0,0.0,None,None,None,Yermoliev,Russia


#### Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [27]:
# Chequeamos los valores nulos de cada columna
df[["revenue","budget"]].isnull().sum()

revenue    6
budget     0
dtype: int64

In [28]:
# Rellenamos los valores nulos de la columan revenue
df["revenue"].fillna(0,inplace=True)

In [29]:
#  Vemos que efectivamente fueron rellenados
df[["revenue","budget"]].isnull().sum()

revenue    0
budget     0
dtype: int64

#### Los valores nulos del campo release date deben eliminarse.

In [30]:
# Creamos un nuevo dataframe con el mismo nombre con los valores de la columna "release_date" que no sean nulos
df = df[df['release_date'].notna()]

In [31]:
df

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,tagline,title,video,vote_average,vote_count,collecion,generos,idioma,compania_productora,pais_produccion
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,"Animation, Comedy, Family",English,Pixar Animation Studios,United States of America
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,None,"Adventure, Fantasy, Family","English, Français","TriStar Pictures, Teitler Film, Interscope Com...",United States of America
2,False,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection,"Romance, Comedy",English,"Warner Bros., Lancaster Gate",United States of America
3,False,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,None,"Comedy, Drama, Romance",English,Twentieth Century Fox Film Corporation,United States of America
4,False,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection,Comedy,English,"Sandollar Productions, Touchstone Pictures",United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,False,0,NaN,30840,tt0102797,en,Robin Hood,"Yet another version of the classic epic, with ...",5.683753,/fQC46NglNiEMZBv5XHoyLuOWoN5.jpg,...,NaN,Robin Hood,False,5.7,26.0,None,"Drama, Action, Romance",English,"Westdeutscher Rundfunk (WDR), Working Title Fi...","Canada, Germany, United Kingdom, United States..."
45462,False,0,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,...,NaN,Century of Birthing,False,9.0,3.0,None,Drama,,Sine Olivia,Philippines
45463,False,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,A deadly game of wits.,Betrayal,False,3.8,6.0,None,"Action, Drama, Thriller",English,American World Pictures,United States of America
45464,False,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,NaN,Satan Triumphant,False,0.0,0.0,None,None,None,Yermoliev,Russia


#### De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.



In [32]:
df["release_date"] = pd.to_datetime(df["release_date"], format="%Y-%m-%d",errors='coerce')


In [33]:
df["release_date"].isnull().sum()

3

In [34]:
df = df[df['release_date'].notna()]

In [35]:
df["release_year"] = df["release_date"].dt.year

#### Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [36]:
df["budget"] = pd.to_numeric(df["budget"])

In [37]:
df["return"] = df["revenue"] / df["budget"]

In [38]:
df["return"].fillna(0,inplace=True)

In [39]:
df['return'] = df['return'].replace([np.inf, -np.inf], 0)

In [40]:
df["return"] = df["return"].round(4)

In [41]:
df["return"]

0        12.4518
1         4.0430
2         0.0000
3         5.0908
4         0.0000
          ...   
45460     0.0000
45462     0.0000
45463     0.0000
45464     0.0000
45465     0.0000
Name: return, Length: 45376, dtype: float64

#### Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [42]:
df.drop(columns=["video","imdb_id","adult","original_title","poster_path","homepage"],inplace=True)

In [43]:
df.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'collecion', 'generos', 'idioma',
       'compania_productora', 'pais_produccion', 'release_year', 'return'],
      dtype='object')

In [44]:
orden = ["id","title","generos","idioma","release_year","release_date","runtime","budget","revenue","return","vote_average","vote_count","popularity","status","tagline","overview","collecion","compania_productora","pais_produccion"]
df = df.reindex(columns=orden)

In [45]:
df

,id,title,generos,idioma,release_year,release_date,runtime,budget,revenue,return,vote_average,vote_count,popularity,status,tagline,overview,collecion,compania_productora,pais_produccion
0,862,Toy Story,"Animation, Comedy, Family",English,1995,1995-10-30,81.0,30000000,373554033.0,12.4518,7.7,5415.0,21.946943,Released,NaN,"Led by Woody, Andy's toys live happily in his ...",Toy Story Collection,Pixar Animation Studios,United States of America
1,8844,Jumanji,"Adventure, Fantasy, Family","English, Français",1995,1995-12-15,104.0,65000000,262797249.0,4.0430,6.9,2413.0,17.015539,Released,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,None,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America
2,15602,Grumpier Old Men,"Romance, Comedy",English,1995,1995-12-22,101.0,0,0.0,0.0000,6.5,92.0,11.7129,Released,Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,Grumpy Old Men Collection,"Warner Bros., Lancaster Gate",United States of America
3,31357,Waiting to Exhale,"Comedy, Drama, Romance",English,1995,1995-12-22,127.0,16000000,81452156.0,5.0908,6.1,34.0,3.859495,Released,Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...",None,Twentieth Century Fox Film Corporation,United States of America
4,11862,Father of the Bride Part II,Comedy,English,1995,1995-02-10,106.0,0,76578911.0,0.0000,5.7,173.0,8.387519,Released,Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,Father of the Bride Collection,"Sandollar Productions, Touchstone Pictures",United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,30840,Robin Hood,"Drama, Action, Romance",English,1991,1991-05-13,104.0,0,0.0,0.0000,5.7,26.0,5.683753,Released,NaN,"Yet another version of the classic epic, with ...",None,"Westdeutscher Rundfunk (WDR), Working Title Fi...","Canada, Germany, United Kingdom, United States..."
45462,111109,Century of Birthing,Drama,,2011,2011-11-17,360.0,0,0.0,0.0000,9.0,3.0,0.178241,Released,NaN,An artist struggles to finish his work while a...,None,Sine Olivia,Philippines
45463,67758,Betrayal,"Action, Drama, Thriller",English,2003,2003-08-01,90.0,0,0.0,0.0000,3.8,6.0,0.903007,Released,A deadly game of wits.,"When one of her hits goes wrong, a professiona...",None,American World Pictures,United States of America
45464,227506,Satan Triumphant,None,None,1917,1917-10-21,87.0,0,0.0,0.0000,0.0,0.0,0.003503,Released,NaN,"In a small town live two brothers, one a minis...",None,Yermoliev,Russia


#### Limpieza de datos

In [46]:
# id duplicados
print(credits.id.value_counts())
print(df.id.value_counts())


141971    3
298721    2
9755      2
10991     2
99080     2
         ..
55135     1
15877     1
72272     1
1549      1
461257    1
Name: id, Length: 45432, dtype: int64
141971    3
97995     2
10991     2
109962    2
119916    2
         ..
40594     1
23104     1
37605     1
91727     1
461257    1
Name: id, Length: 45346, dtype: int64


In [47]:
credits.where(credits["id"] == 141971).dropna()

,id,actores,director
13261,141971.0,"Petteri Summanen, Ismo Kallio, Eppu Salminen, ...",JP Siili
13375,141971.0,"Petteri Summanen, Ismo Kallio, Eppu Salminen, ...",JP Siili
16764,141971.0,"Petteri Summanen, Ismo Kallio, Eppu Salminen, ...",JP Siili


In [48]:
credits = credits.drop_duplicates(subset='id', keep='first')
df = df.drop_duplicates(subset='id', keep='first')

In [49]:
# id duplicados
print(credits.id.value_counts())
print(df.id.value_counts())


862       1
74458     1
296206    1
107308    1
16247     1
         ..
32985     1
44399     1
10138     1
32084     1
461257    1
Name: id, Length: 45432, dtype: int64
862       1
202198    1
124026    1
300168    1
132316    1
         ..
18406     1
39053     1
37169     1
24805     1
461257    1
Name: id, Length: 45346, dtype: int64


### Union de los dataframes

In [50]:
print(df.info())
print(credits.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 45346 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   45346 non-null  object        
 1   title                45346 non-null  object        
 2   generos              42962 non-null  object        
 3   idioma               41580 non-null  object        
 4   release_year         45346 non-null  int64         
 5   release_date         45346 non-null  datetime64[ns]
 6   runtime              45100 non-null  float64       
 7   budget               45346 non-null  int64         
 8   revenue              45346 non-null  float64       
 9   return               45346 non-null  float64       
 10  vote_average         45346 non-null  float64       
 11  vote_count           45346 non-null  float64       
 12  popularity           45346 non-null  object        
 13  status               45266 non-

In [51]:
df["id"] = df["id"].astype("Int64")

In [52]:
df = df.join(credits.set_index('id'), on='id')

In [53]:
df.reset_index(inplace=True)

In [54]:
df.drop(columns="index",inplace=True)

In [55]:
df.columns

Index(['id', 'title', 'generos', 'idioma', 'release_year', 'release_date',
       'runtime', 'budget', 'revenue', 'return', 'vote_average', 'vote_count',
       'popularity', 'status', 'tagline', 'overview', 'collecion',
       'compania_productora', 'pais_produccion', 'actores', 'director'],
      dtype='object')

In [56]:
df.rename(columns={"title":"titulo","release_year":"año_estreno","release_date":"fecha_estreno","runtime":"duracion (minutos)","budget":"presupuesto","revenue":"recaudaccion","return":"retorno","vote_average":"promedio_votos","vote_count":"cantidad_votos","popularity":"popularidad","status":"estado","tagline":"frase","overview":"resumen"},inplace=True)

In [57]:
df

,id,titulo,generos,idioma,año_estreno,fecha_estreno,duracion (minutos),presupuesto,recaudaccion,retorno,...,cantidad_votos,popularidad,estado,frase,resumen,collecion,compania_productora,pais_produccion,actores,director
0,862,Toy Story,"Animation, Comedy, Family",English,1995,1995-10-30,81.0,30000000,373554033.0,12.4518,...,5415.0,21.946943,Released,NaN,"Led by Woody, Andy's toys live happily in his ...",Toy Story Collection,Pixar Animation Studios,United States of America,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",John Lasseter
1,8844,Jumanji,"Adventure, Fantasy, Family","English, Français",1995,1995-12-15,104.0,65000000,262797249.0,4.0430,...,2413.0,17.015539,Released,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,None,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Joe Johnston
2,15602,Grumpier Old Men,"Romance, Comedy",English,1995,1995-12-22,101.0,0,0.0,0.0000,...,92.0,11.7129,Released,Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,Grumpy Old Men Collection,"Warner Bros., Lancaster Gate",United States of America,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Howard Deutch
3,31357,Waiting to Exhale,"Comedy, Drama, Romance",English,1995,1995-12-22,127.0,16000000,81452156.0,5.0908,...,34.0,3.859495,Released,Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...",None,Twentieth Century Fox Film Corporation,United States of America,"Whitney Houston, Angela Bassett, Loretta Devin...",Forest Whitaker
4,11862,Father of the Bride Part II,Comedy,English,1995,1995-02-10,106.0,0,76578911.0,0.0000,...,173.0,8.387519,Released,Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,Father of the Bride Collection,"Sandollar Productions, Touchstone Pictures",United States of America,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45341,30840,Robin Hood,"Drama, Action, Romance",English,1991,1991-05-13,104.0,0,0.0,0.0000,...,26.0,5.683753,Released,NaN,"Yet another version of the classic epic, with ...",None,"Westdeutscher Rundfunk (WDR), Working Title Fi...","Canada, Germany, United Kingdom, United States...","Patrick Bergin, Uma Thurman, David Morrissey, ...",John Irvin
45342,111109,Century of Birthing,Drama,,2011,2011-11-17,360.0,0,0.0,0.0000,...,3.0,0.178241,Released,NaN,An artist struggles to finish his work while a...,None,Sine Olivia,Philippines,"Angel Aquino, Perry Dizon, Hazel Orencio, Joel...",Lav Diaz
45343,67758,Betrayal,"Action, Drama, Thriller",English,2003,2003-08-01,90.0,0,0.0,0.0000,...,6.0,0.903007,Released,A deadly game of wits.,"When one of her hits goes wrong, a professiona...",None,American World Pictures,United States of America,"Erika Eleniak, Adam Baldwin, Julie du Page, Ja...",Mark L. Lester
45344,227506,Satan Triumphant,None,None,1917,1917-10-21,87.0,0,0.0,0.0000,...,0.0,0.003503,Released,NaN,"In a small town live two brothers, one a minis...",None,Yermoliev,Russia,"Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav...",Yakov Protazanov


#### CREACION DE DATASETS Y FUNCIONES PARA LA API

Primera funcion
* def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

In [58]:
peliculas_mes = df[["id","fecha_estreno"]]

In [59]:
peliculas_mes["mes"] = peliculas_mes["fecha_estreno"].dt.month_name()

In [60]:
peliculas_mes.drop(columns="fecha_estreno",inplace=	True)

In [61]:
peliculas_mes

,id,mes
0,862,October
1,8844,December
2,15602,December
3,31357,December
4,11862,February
...,...,...
45341,30840,May
45342,111109,November
45343,67758,August
45344,227506,October


In [62]:
peliculas_mes = peliculas_mes.groupby("mes").size().reset_index(name="cantidad")

In [63]:
peliculas_mes.replace({"April":"Abril","August":"Agosto","December":"Diciembre","February":"Febrero","January":"Enero","July":"Julio","June":"Junio","March":"Marzo","May":"Mayo","November":"Noviembre","October":"Octubre","September":"Septiembre"},inplace=True)

In [64]:
def pelicula_mes(mes: str):
    try:
        
        cantidad = peliculas_mes.loc[peliculas_mes["mes"] == mes, "cantidad"].dropna().values
        return {"mes": mes, "cantidad": cantidad[0]}
    
    except Exception as e:
        
        return "El mes ingresado no es valido"


In [65]:
pelicula_mes("Enero")

{'mes': 'Enero', 'cantidad': 5909}

In [66]:
peliculas_mes.to_csv("API\peliculas_mes.csv",index=False)

def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

In [67]:
peliculas_dia = df[["id","fecha_estreno"]]

In [68]:
peliculas_dia["dia"] = peliculas_dia["fecha_estreno"].dt.day_name()

In [69]:
peliculas_dia.drop(columns="fecha_estreno",inplace=True)

In [70]:
peliculas_dia

,id,dia
0,862,Monday
1,8844,Friday
2,15602,Friday
3,31357,Friday
4,11862,Friday
...,...,...
45341,30840,Monday
45342,111109,Thursday
45343,67758,Friday
45344,227506,Sunday


In [71]:
peliculas_dia = peliculas_dia.groupby("dia").size().reset_index(name="cantidad")

In [72]:
peliculas_dia.replace({"Friday":"Viernes","Monday":"Lunes","Saturday":"Sabado","Sunday":"Domingo","Thursday":"Jueves","Tuesday":"Martes","Wednesday":"Miercoles"},inplace=True)

In [73]:
peliculas_dia

,dia,cantidad
0,Viernes,13902
1,Lunes,3500
2,Sabado,5149
3,Domingo,3608
4,Jueves,7520
5,Martes,4640
6,Miercoles,7027


In [74]:
def pelicula_dia(dia: str):
    try:
        dia = dia.capitalize()
        cantidad = peliculas_dia.loc[peliculas_dia["dia"] == dia, "cantidad"].values
        
        return {"dia": dia, "cantidad": cantidad[0]}
    except Exception as e:
        
        return "El dia ingresado no es valido"

In [75]:
pelicula_dia("Domingo")

{'dia': 'Domingo', 'cantidad': 3608}

In [76]:
peliculas_dia.to_csv("API\peliculas_dia.csv",index=False)

def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
                    Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de X

In [77]:
score_titulo = df[["titulo","año_estreno","popularidad"]]

In [78]:
tit_score = score_titulo.loc[score_titulo["titulo"]=="Toy Story"].values
print(tit_score)

[['Toy Story' 1995 21.946943]]


In [79]:
def titulo_score(pelicula: str):
    try:
        pelicula_info = score_titulo.loc[score_titulo["titulo"] == pelicula]
        tit_score = pelicula_info.values[0]
        return f"La película {pelicula} fue estrenada en el año {tit_score[1]} con un score de {tit_score[2]}"
    except Exception as e:
        return "El título ingresado no se encuentra"


In [80]:
titulo_score("Guardians of the Galaxy")

'La película Guardians of the Galaxy fue estrenada en el año 2014 con un score de 53.291601'

In [81]:
score_titulo.to_csv("API\score_titulo.csv")

def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
                    Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio

In [82]:
votos_titulos = df[["titulo","cantidad_votos","promedio_votos","año_estreno"]]

In [83]:
votos_titulos = votos_titulos.where(votos_titulos["cantidad_votos"]>=2000).dropna()

In [84]:
def titulo_votos(pelicula:str):
    try:
        peli_votos = votos_titulos.loc[votos_titulos["titulo"]==pelicula].values
        return f"La pelicula {pelicula} tuvo {int(peli_votos[0][1])} cantidad de votos, con un valor promedio de cada voto de {peli_votos[0][2]},{int(peli_votos[0][3])}"
    except Exception:
        return ("La pelicula no supera la cantidad ed 2000 votos")

In [85]:
titulo_votos("Toy Story")

'La pelicula Toy Story tuvo 5415 cantidad de votos, con un valor promedio de cada voto de 7.7,1995'

In [86]:
votos_titulos.to_csv("API\ votos_titulos.csv")

def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
                    Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

In [87]:
nombre_actor = df[["id","actores","retorno"]]

In [88]:
nombre_actor["actores"].isnull().sum()

2349

In [89]:
nombre_actor.dropna(inplace=True)

In [90]:
def actor_datos(actor:str):
    retorno_total = 0
    cant_peliculas = 0
    for i, fila in nombre_actor.iterrows():
        if actor in fila["actores"]:
            retorno_total += fila["retorno"]
            cant_peliculas += 1
    retorno_promedio = retorno_total / cant_peliculas if cant_peliculas > 0 else 0
    
    return f"El actor {actor} a participado en {cant_peliculas} filmaciones, recaudando un retorno de {retorno_total} y un promedio de retorno de  {retorno_promedio}"


In [91]:
actor_datos("Chris Pratt")

'El actor Chris Pratt a participado en 25 filmaciones, recaudando un retorno de 59.680899999999994 y un promedio de retorno de  2.3872359999999997'

In [92]:
nombre_actor.to_csv("API\ nombre_actor.csv")

def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [93]:
nombre_director = df[["titulo","fecha_estreno","retorno","presupuesto","recaudaccion","director"]]

In [94]:
nombre_director["director"].replace("",None,inplace=True)

In [95]:
nombre_director["director"].isnull().sum()

836

In [96]:
nombre_director["director"].value_counts()

John Ford                   63
Michael Curtiz              61
Alfred Hitchcock            52
Werner Herzog               52
Georges Méliès              51
                            ..
Rob Meltzer                  1
Ignacio Ferreras             1
Dylan Mohan Gray             1
Spike Jonze, Lance Bangs     1
Daisy Asquith                1
Name: director, Length: 18828, dtype: int64

In [97]:
nombre_director.dropna(inplace=True)

In [98]:
def director(director: str):
    try:
        dire = nombre_director.loc[nombre_director["director"] == director]
        fecha_estreno = [date.strftime("%Y-%m-%d") for date in dire["fecha_estreno"]]
        return {"director":director,"retorno_total_director":dire["retorno"].sum(),"peliculas":list(dire["titulo"]),"año":fecha_estreno,
                "retorno_pelicula":list(dire["retorno"]),"budget_pelicula":list(dire["presupuesto"]),"revenue_pelicula":list(dire["recaudaccion"])}

    except Exception as e:
        return f"Hubo un error:{e}"


In [102]:
director("Quentin Tarantino")

{'director': 'Quentin Tarantino',
 'retorno_total_director': 66.7225,
 'peliculas': ['Pulp Fiction',
  'Reservoir Dogs',
  'Jackie Brown',
  'Kill Bill: Vol. 1',
  'Kill Bill: Vol. 2',
  'Death Proof',
  'Inglourious Basterds',
  'Django Unchained',
  'The Hateful Eight',
  "My Best Friend's Birthday"],
 'año': ['1994-09-10',
  '1992-09-02',
  '1997-12-24',
  '2003-10-10',
  '2004-04-16',
  '2007-05-21',
  '2009-08-18',
  '2012-12-25',
  '2015-12-25',
  '1987-01-01'],
 'retorno_pelicula': [26.7411,
  12.2175,
  3.3061,
  6.0316,
  5.072,
  1.0015,
  4.559,
  4.2537,
  3.54,
  0.0],
 'budget_pelicula': [8000000,
  1200000,
  12000000,
  30000000,
  30000000,
  25000000,
  70000000,
  100000000,
  44000000,
  0],
 'revenue_pelicula': [213928762.0,
  14661007.0,
  39673162.0,
  180949000.0,
  152159461.0,
  25037897.0,
  319131050.0,
  425368238.0,
  155760117.0,
  0.0]}

In [101]:
nombre_director.to_csv("API\ nombre_director.csv")